분석에 이용한 자료는 2020-12-27 에 수집하였습니다.

# 패키지 설치

In [16]:
import csv
import json
import pandas as pd
import requests

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import plotly.offline as py

pd.options.plotting.backend = "plotly"
pio.renderers.default = "notebook_connected"

# 데이터 불러오기

In [17]:
spposition_url = requests.get('https://static.api.nexon.co.kr/fifaonline4/latest/spposition.json')
spposition_parsed_data = spposition_url.json()
spposition = pd.DataFrame(spposition_parsed_data)

In [24]:
spposition

,spposition,desc
0,0,GK
1,1,SW
2,2,RWB
3,3,RB
4,4,RCB
5,5,CB
6,6,LCB
7,7,LB
8,8,LWB
9,9,RDM


In [25]:
print("position at %s" %(spposition['desc'][1]))

position at SW


In [2]:
messi = pd.read_csv('messi.csv',
                   index_col = 0)

ronaldo = pd.read_csv('ronaldo.csv',
                     index_col = 0)

neymar = pd.read_csv('neymar.csv',
                    index_col = 0)

# 각 선수의 주요 기용 포지션 확인

랭커 유저는 메시, 호날두, 네이마르를 어느 포지션에서 가장 많이 활용할까? 

In [3]:
physical_theta = ['effectiveShoot', 'assist', 'goal', 'block', 'tackle']
technical_theta = ['passRate', 'dribbleRate']

## 메시

In [4]:
messi_top3 = messi.sort_values(by = 'matchCount', ascending = False).head(3)
messi_top3.reset_index(inplace = True)
messi_top3.drop(['index'], axis=1, inplace = True)
messi_top3

,spId,spPosition,shoot,effectiveShoot,assist,goal,dribble,dribbleTry,dribbleSuccess,passTry,passSuccess,block,tackle,matchCount,passRate,dribbleRate
0,206158023,21,1.53333,1.26667,0.73333,0.60000,33.60000,21.40000,19.26667,22.46667,17.20000,0.0,0.26667,15,0.765579,0.900312
1,206158023,26,1.83333,1.66667,0.50000,1.16667,41.33333,10.83333,8.83333,14.33333,13.00000,0.0,0.83333,6,0.906977,0.815385
2,206158023,6,0.00000,0.00000,0.00000,0.00000,2.66667,3.33333,2.66667,5.66667,3.66667,0.0,1.00000,3,0.647059,0.800002


### 메시 선호도 1위 포지션

In [43]:
messi_1st = pd.DataFrame(messi_top3.loc[0:0], columns = ['effectiveShoot', 'assist', 'goal', 'block', 'tackle'])
messi_1st

,effectiveShoot,assist,goal,block,tackle
0,1.26667,0.73333,0.6,0.0,0.26667


In [44]:
messi_RM_fig = px.line_polar(r = [messi_1st.iloc[0][0], messi_1st.iloc[0][1], messi_1st.iloc[0][2], messi_1st.iloc[0][3], messi_1st.iloc[0][4]],
                   theta = physical_theta, line_close = True)

messi_RM_fig.update_traces(fill = 'toself')
messi_RM_fig.update_layout(
                title = {
                    'text' : '메시, 포지션 %s' %(spposition['desc'][messi_top3['spPosition'][0]]),
                    'y': 0.97,
                    'x': 0.5,
                    'xanchor': 'center'

                })
messi_RM_fig.show()

### 메시 선호도 2위 포지션

In [45]:
messi_2nd = pd.DataFrame(messi_top3.loc[1:1], columns = ['effectiveShoot', 'assist', 'goal', 'block', 'tackle'])
messi_2nd

,effectiveShoot,assist,goal,block,tackle
1,1.66667,0.5,1.16667,0.0,0.83333


In [46]:
messi_LF_fig = px.line_polar(r = [messi_2nd.iloc[0][0], messi_2nd.iloc[0][1], messi_2nd.iloc[0][2], messi_2nd.iloc[0][3], messi_2nd.iloc[0][4]],
                   theta = physical_theta, line_close = True)

messi_LF_fig.update_traces(fill = 'toself')
messi_LF_fig.update_layout(
                title = {
                    'text' : '메시, 포지션 %s' %(spposition['desc'][messi_top3['spPosition'][1]]),
                    'y': 0.97,
                    'x': 0.5,
                    'xanchor': 'center'

                })
messi_LF_fig.show()

### 메시 선호도 3위 포지션

In [47]:
messi_3rd = pd.DataFrame(messi_top3.loc[2:2], columns = ['effectiveShoot', 'assist', 'goal', 'block', 'tackle'])
messi_3rd

,effectiveShoot,assist,goal,block,tackle
2,0.0,0.0,0.0,0.0,1.0


In [48]:
messi_LM_fig = px.line_polar(r = [messi_3rd.iloc[0][0], messi_3rd.iloc[0][1], messi_3rd.iloc[0][2], messi_3rd.iloc[0][3], messi_3rd.iloc[0][4]],
                   theta = physical_theta, line_close = True)

messi_LM_fig.update_traces(fill = 'toself')
messi_LM_fig.update_layout(
                title = {
                    'text' : '메시, 포지션 %s' %(spposition['desc'][messi_top3['spPosition'][2]]),
                    'y': 0.97,
                    'x': 0.5,
                    'xanchor': 'center'

                })
messi_LM_fig.show()

In [49]:
messi_fig = go.Figure()

messi_fig.add_trace(go.Scatterpolar(
                        r = [messi_1st.iloc[0][0], messi_1st.iloc[0][1], messi_1st.iloc[0][2], messi_1st.iloc[0][3], messi_1st.iloc[0][4]],
                        theta = physical_theta,
                        fill = 'toself',
                        name = '메시, 포지션 %s' %(spposition['desc'][messi_top3['spPosition'][0]])))

messi_fig.add_trace(go.Scatterpolar(
                        r = [messi_2nd.iloc[0][0], messi_2nd.iloc[0][1], messi_2nd.iloc[0][2], messi_2nd.iloc[0][3], messi_2nd.iloc[0][4]],
                        theta = physical_theta,
                        fill = 'toself',
                        name = '메시, 포지션 %s' %(spposition['desc'][messi_top3['spPosition'][1]])))

messi_fig.add_trace(go.Scatterpolar(
                        r = [messi_3rd.iloc[0][0], messi_3rd.iloc[0][1], messi_3rd.iloc[0][2], messi_3rd.iloc[0][3], messi_3rd.iloc[0][4]],
                        theta = physical_theta,
                        fill = 'toself',
                        name = '메시, 포지션 %s' %(spposition['desc'][messi_top3['spPosition'][2]])))


messi_fig.update_layout(
                title = {
                    'text' : '메시, 포지션별 피지컬 능력치',
                    'y': 0.97,
                    'x': 0.5,
                    'xanchor': 'center'

                })

messi_fig.show()

## 호날두

In [51]:
ronaldo_top3 = ronaldo.sort_values(by = 'matchCount', ascending = False).head(3)
ronaldo_top3.reset_index(inplace = True)
ronaldo_top3.drop(['index'], axis=1, inplace = True)
ronaldo_top3

,spId,spPosition,shoot,effectiveShoot,assist,goal,dribble,dribbleTry,dribbleSuccess,passTry,passSuccess,block,tackle,matchCount,passRate,dribbleRate
0,206020801,18,1.64286,1.14286,0.21429,0.64286,48.00000,15.42857,14.21429,18.85714,17.50000,0.0,0.28571,14,0.928030,0.921297
1,206020801,16,0.50000,0.33333,0.08333,0.16667,49.16667,7.41667,5.83333,10.00000,7.41667,0.0,0.83333,12,0.741667,0.786516
2,206020801,26,2.09091,1.36364,0.00000,0.18182,32.72727,10.00000,8.18182,12.72727,10.27273,0.0,0.18182,11,0.807143,0.818182


In [54]:
ronaldo_radar = ronaldo_top3[['effectiveShoot', 'assist', 'goal', 'tackle', 'passRate', 'dribbleRate']]
ronaldo_radar

,effectiveShoot,assist,goal,tackle,passRate,dribbleRate
0,1.14286,0.21429,0.64286,0.28571,0.928030,0.921297
1,0.33333,0.08333,0.16667,0.83333,0.741667,0.786516
2,1.36364,0.00000,0.18182,0.18182,0.807143,0.818182


## 네이마르

In [53]:
neymar_top3 = neymar.sort_values(by = 'matchCount', ascending = False).head(3)
neymar_top3.reset_index(inplace = True)
neymar_top3.drop(['index'], axis=1, inplace = True)
neymar_top3

,spId,spPosition,shoot,effectiveShoot,assist,goal,dribble,dribbleTry,dribbleSuccess,passTry,passSuccess,block,tackle,matchCount,passRate,dribbleRate
0,207190871,27,0.46154,0.38462,0.23077,0.15385,0.00000,3.00000,2.61538,4.15385,3.38462,0,0.30769,13,0.814815,0.871793
1,207190871,16,0.44444,0.44444,0.22222,0.22222,22.55556,8.55556,7.44444,10.22222,8.00000,0,0.55556,9,0.782609,0.870129
2,207190871,12,0.00000,0.00000,0.20000,0.00000,0.00000,6.00000,4.60000,6.00000,3.80000,0,0.00000,5,0.633333,0.766667


In [24]:
neymar_radar = ronaldo_top3[['effectiveShoot', 'assist', 'goal', 'tackle', 'passRate', 'dribbleRate']]
neymar_radar

,effectiveShoot,assist,goal,tackle,passRate,dribbleRate
0,1.05,0.20000,0.50000,0.20000,0.875000,0.783920
1,1.25,0.10000,0.60000,0.20000,0.901961,0.811404
2,1.00,0.21429,0.28571,0.14286,0.908629,0.852564


In [2]:
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

In [3]:
with open('data.pickle', 'rb') as f:
    fifa = pickle.load(f)

In [4]:
spId_url = requests.get('https://static.api.nexon.co.kr/fifaonline4/latest/spid.json')
spId_parsed_data = spId_url.json()
spId = pd.DataFrame(spId_parsed_data)

In [5]:
spposition_url = requests.get('https://static.api.nexon.co.kr/fifaonline4/latest/spposition.json')
spposition_parsed_data = spposition_url.json()
spposition = pd.DataFrame(spposition_parsed_data)

- ```assist``` : 평균 어시스트 수
- ```blcok``` : 평균 블락 성공 수
- ```dribble``` : 평균 드리블 거리(야드)
- ```effectiveShoot``` : 평균 유효 슛 수
- ```goal``` : 평균 득점 수
- ```matchCount``` : 해당 포지션으로 경기 참여한 횟수
- ```passSuccess``` : 평균 패스 성공 수
- ```tackle``` : 평균 태클 성공 수

In [6]:
fifa.head()

,spId,spPosition,assist,block,dribble,effectiveShoot,goal,matchCount,tackle,passRate
0,101000001,0,0.0,0.0,0.0,0.0,0.0,4,0.0,0.928571
1,101000001,28,0.0,0.0,0.0,0.0,0.0,2,0.0,NaN
2,101000195,4,0.0,0.0,0.0,0.0,0.0,1,0.0,1.000000
3,101000195,12,0.5,0.0,85.0,2.0,0.0,2,0.0,0.848485
4,101000195,14,2.0,0.0,0.0,0.0,0.0,1,0.0,0.928571


## 주요 포지션별 선수 스탯

오각형 모양의 선수 스탯 그래프 그리기 연습

In [7]:
categories = ['assist','block','effectiveShoot','tackle']

## 공격수

### 스트라이커(ST)

In [8]:
st = fifa[(fifa.spPosition == 25) & (fifa.matchCount == 20)]
st_top3 = st.sort_values(by = 'effectiveShoot', ascending = False).head(3)
st_top3.reset_index( inplace = True )
st_top3.drop(['index'], axis=1, inplace = True)
st_top3

,spId,spPosition,assist,block,dribble,effectiveShoot,goal,matchCount,tackle,passRate
0,101031432,25,0.15,0.0,25.15,3.95,1.90,20,0.15,0.875000
1,207000051,25,0.05,0.0,45.85,3.05,0.95,20,0.15,0.965714
2,215224179,25,0.20,0.0,54.35,2.30,1.15,20,0.25,0.922414


In [9]:
st_topSp = []

for i in st_top3['spId'] :
    st_topSp.append(list(spId[spId['id'] == i].name))
    
st_topSp = sum(st_topSp, [])

In [8]:
from plotly.offline import plot
st_fig = go.Figure()

st_fig.add_trace(go.Scatterpolar(
      r=[st_top3.loc[0]['assist'], st_top3.loc[0]['block'], 
         st_top3.loc[0]['effectiveShoot'], st_top3.loc[0]['tackle']],
      theta=categories,
      fill='toself',
      name=st_topSp[0]
))

st_fig.add_trace(go.Scatterpolar(
      r=[st_top3.loc[1]['assist'], st_top3.loc[1]['block'], 
         st_top3.loc[1]['effectiveShoot'], st_top3.loc[1]['tackle']],
      theta=categories,
      fill='toself',
      name=st_topSp[1]
))

st_fig.add_trace(go.Scatterpolar(
      r=[st_top3.loc[2]['assist'], st_top3.loc[2]['block'], 
         st_top3.loc[2]['effectiveShoot'], st_top3.loc[2]['tackle']],
      theta=categories,
      fill='toself',
      name=st_topSp[2]
))



st_fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 4.5]
    )),
  showlegend=True,
    title = {
        'text': '주요 스트라이커 운용 비교',
        'x': 0.45, 
        'y': 0.9,
    }
)

st_fig.show()

NameError: name 'st_top3' is not defined

### 센터 포워드(CF)

In [11]:
cf = fifa[(fifa.spPosition == 21) & (fifa.matchCount == 20)]
cf_top3 = cf.sort_values(by = 'effectiveShoot', ascending = False).head(3)
cf_top3.reset_index( inplace = True )
cf_top3.drop(['index'], axis=1, inplace = True)
cf_top3

,spId,spPosition,assist,block,dribble,effectiveShoot,goal,matchCount,tackle,passRate
0,101190045,21,0.40,0.05,32.75,2.20,0.85,20,0.25,0.845921
1,219242444,21,0.20,0.00,3.95,2.15,0.45,20,0.10,0.884868
2,295191910,21,0.35,0.00,34.85,1.85,0.85,20,0.35,0.865031


In [12]:
cf_topSp = []

for i in cf_top3['spId'] :
    cf_topSp.append(list(spId[spId['id'] == i].name))
    
cf_topSp = sum(cf_topSp, [])

In [13]:
cf_fig = go.Figure()



cf_fig.add_trace(go.Scatterpolar(
      r=[cf_top3.loc[0]['assist'], cf_top3.loc[0]['block'], 
         cf_top3.loc[0]['effectiveShoot'], cf_top3.loc[0]['tackle']],
      theta=categories,
      fill='toself',
      name=cf_topSp[0]
))

cf_fig.add_trace(go.Scatterpolar(
      r=[cf_top3.loc[1]['assist'], cf_top3.loc[1]['block'], 
         cf_top3.loc[1]['effectiveShoot'], cf_top3.loc[1]['tackle']],
      theta=categories,
      fill='toself',
      name=cf_topSp[1]
))

cf_fig.add_trace(go.Scatterpolar(
      r=[cf_top3.loc[2]['assist'], cf_top3.loc[2]['block'], 
         cf_top3.loc[2]['effectiveShoot'], cf_top3.loc[2]['tackle']],
      theta=categories,
      fill='toself',
      name=cf_topSp[2]
))



cf_fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 2.5]
    )),
  showlegend=True,
    title = {
        'text': '주요 센터 포워드 운용 비교',
        'x': 0.46, 
        'y': 0.9
    }
)

cf_fig.show()

### 중앙 미드필더(CM)

In [14]:
cm = fifa[(fifa.spPosition == 14) & (fifa.matchCount == 20)]
cm_top3 = cm.sort_values(by = 'passRate', ascending = False).head(3)
cm_top3.reset_index( inplace = True )
cm_top3.drop(['index'], axis=1, inplace = True)
cm_top3

,spId,spPosition,assist,block,dribble,effectiveShoot,goal,matchCount,tackle,passRate
0,206233419,14,0.2,0.05,54.50,0.60,0.15,20,0.45,0.952218
1,207177003,14,0.0,0.00,3.10,0.35,0.05,20,0.40,0.947552
2,221156519,14,0.3,0.00,27.55,0.60,0.25,20,0.95,0.945161


In [15]:
cm_topSp = []

for i in cm_top3['spId'] :
    cm_topSp.append(list(spId[spId['id'] == i].name))
    
cm_topSp = sum(cm_topSp, [])

In [16]:
cm_fig = go.Figure()



cm_fig.add_trace(go.Scatterpolar(
      r=[cm_top3.loc[0]['assist'], cm_top3.loc[0]['block'], 
         cm_top3.loc[0]['effectiveShoot'], cm_top3.loc[0]['tackle']],
      theta=categories,
      fill='toself',
      name=cm_topSp[0]
))

cm_fig.add_trace(go.Scatterpolar(
      r=[cm_top3.loc[1]['assist'], cm_top3.loc[1]['block'], 
         cm_top3.loc[1]['effectiveShoot'], cm_top3.loc[1]['tackle']],
      theta=categories,
      fill='toself',
      name=cm_topSp[1]
))

cm_fig.add_trace(go.Scatterpolar(
      r=[cm_top3.loc[2]['assist'], cm_top3.loc[2]['block'], 
         cm_top3.loc[2]['effectiveShoot'], cm_top3.loc[2]['tackle']],
      theta=categories,
      fill='toself',
      name=cm_topSp[2]
))



cm_fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 1.2]
    )),
  showlegend=True,
    title = {
        'text': '주요 중앙 미드필더 운용 비교',
        'x': 0.46, 
        'y': 0.9
    }
)

cm_fig.show()

## 수비수

### 윙어(LW)

In [17]:
lw = fifa[(fifa.spPosition == 27) & (fifa.matchCount == 20)]
lw_top3 = lw.sort_values(by = 'passRate', ascending = False).head(3)
lw_top3.reset_index( inplace = True )
lw_top3.drop(['index'], axis=1, inplace = True)
lw_top3

,spId,spPosition,assist,block,dribble,effectiveShoot,goal,matchCount,tackle,passRate
0,210189596,27,0.20,0.0,28.75,0.30,0.20,20,0.15,0.941176
1,221208722,27,0.20,0.0,35.75,0.40,0.20,20,0.20,0.936047
2,206184267,27,0.25,0.0,69.05,0.65,0.15,20,0.20,0.931624


In [18]:
lw_topSp = []

for i in lw_top3['spId'] :
    lw_topSp.append(list(spId[spId['id'] == i].name))
    
lw_topSp = sum(lw_topSp, [])

In [19]:
lw_fig = go.Figure()



lw_fig.add_trace(go.Scatterpolar(
      r=[lw_top3.loc[0]['assist'], lw_top3.loc[0]['block'], 
         lw_top3.loc[0]['effectiveShoot'], lw_top3.loc[0]['tackle']],
      theta=categories,
      fill='toself',
      name=lw_topSp[0]
))

lw_fig.add_trace(go.Scatterpolar(
      r=[lw_top3.loc[1]['assist'], lw_top3.loc[1]['block'], 
         lw_top3.loc[1]['effectiveShoot'], lw_top3.loc[1]['tackle']],
      theta=categories,
      fill='toself',
      name=lw_topSp[1]
))

lw_fig.add_trace(go.Scatterpolar(
      r=[lw_top3.loc[2]['assist'], lw_top3.loc[2]['block'], 
         lw_top3.loc[2]['effectiveShoot'], lw_top3.loc[2]['tackle']],
      theta=categories,
      fill='toself',
      name=lw_topSp[2]
))



lw_fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 0.8]
    )),
  showlegend=True,
    title = {
        'text': '주요 윙어 운용 비교',
        'x': 0.46, 
        'y': 0.9
    }
)

lw_fig.show()

## 번외, 특정 선수(포그바)의 최적 포지션 찾기

폴 포그바 선수는 유저의 선호에 따라 다양한 포지션에서 활용 가능한 '1티어' 선수이다. 랭커 유저의 플레이 기록을 바탕으로 해당 선수를 어떤 포지션에 이용하는 것이 좋을지 분석한다. 분석할 포지션은 CM, CAM, CF이다.

In [20]:
pg = spId[spId['id'] == 210195864]
pg.reset_index( inplace = True )
pg.drop(['index'], axis=1, inplace = True)
pg

C:\Users\eunee\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,id,name
0,210195864,폴 포그바


In [21]:
pogba = fifa[(fifa['spId'] == 210195864) & (fifa['matchCount'] == 20)]
pogba.reset_index( inplace = True )
pogba.drop(['index'], axis=1, inplace = True)

In [22]:
pogba

,spId,spPosition,assist,block,dribble,effectiveShoot,goal,matchCount,tackle,passRate
0,210195864,9,0.15,0.10,11.75,0.30,0.05,20,0.45,0.912892
1,210195864,10,0.15,0.00,24.70,0.40,0.05,20,0.75,0.916107
2,210195864,11,0.30,0.05,20.45,0.85,0.25,20,0.80,0.910891
3,210195864,12,0.25,0.10,9.25,0.15,0.05,20,0.35,0.919786
4,210195864,13,0.15,0.00,31.60,0.90,0.35,20,0.65,0.872131
5,210195864,14,0.20,0.00,10.10,0.65,0.20,20,0.65,0.915129
6,210195864,15,0.10,0.00,9.85,0.75,0.45,20,0.65,0.893281
7,210195864,16,0.20,0.15,6.30,0.85,0.35,20,0.80,0.855721
8,210195864,17,0.15,0.00,42.50,0.50,0.10,20,0.35,0.858268
9,210195864,18,0.30,0.00,38.55,1.05,0.60,20,0.40,0.868056


In [23]:
pg_cm = pogba[pogba.spPosition == 14]
pg_cm.reset_index( inplace = True )
pg_cm.drop(['index'], axis=1, inplace = True)

pg_cam = pogba[pogba.spPosition == 18]
pg_cam.reset_index( inplace = True )
pg_cam.drop(['index'], axis=1, inplace = True)

pg_cf = pogba[pogba.spPosition == 21]
pg_cf.reset_index( inplace = True )
pg_cf.drop(['index'], axis=1, inplace = True)

In [24]:
pg_fig = go.Figure()

pg_fig.add_trace(go.Scatterpolar(
      r=[pg_cm.loc[0]['assist'], pg_cm.loc[0]['block'], 
         pg_cm.loc[0]['effectiveShoot'], pg_cm.loc[0]['tackle']],
      theta=categories,
      fill='toself',
      name=spposition[spposition['spposition'] == 14].desc.loc[14]
))

pg_fig.add_trace(go.Scatterpolar(
      r=[pg_cam.loc[0]['assist'], pg_cam.loc[0]['block'], 
         pg_cam.loc[0]['effectiveShoot'], pg_cam.loc[0]['tackle']],
      theta=categories,
      fill='toself',
      name=spposition[spposition['spposition'] == 18].desc.loc[18]
))

pg_fig.add_trace(go.Scatterpolar(
      r=[pg_cf.loc[0]['assist'], pg_cf.loc[0]['block'], 
         pg_cf.loc[0]['effectiveShoot'], pg_cf.loc[0]['tackle']],
      theta=categories,
      fill='toself',
      name=spposition[spposition['spposition'] == 21].desc.loc[21]
))


pg_fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 1.2]
    )),
  showlegend=True,
    title = {
        'text': '폴 포그바 포지션 비교',
        'x': 0.48, 
        'y': 0.9
    }
)

pg_fig.show()

## Plotly 업로드

In [25]:
username = 'euneestella'
api_key = 'sqYACHNXEMAaMS6ZB2ZR'
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

In [26]:
import chart_studio.plotly as py
py.plot(st_fig, filename = 'plotly-st', auto_open=False)

'https://plotly.com/~euneestella/1/'

In [27]:
py.plot(cf_fig, filename = 'plotly-cf', auto_open=False)

'https://plotly.com/~euneestella/5/'

In [28]:
py.plot(cm_fig, filename = 'plotly-cm', auto_open=False)

'https://plotly.com/~euneestella/8/'

In [29]:
py.plot(lw_fig, filename = 'plotly-lw', auto_open=False)

'https://plotly.com/~euneestella/13/'

In [30]:
py.plot(pg_fig, filename = 'plotly-pg', auto_open=False)

'https://plotly.com/~euneestella/15/'

### references
- https://plot.ly/python/radar-chart/
- https://wooiljeong.github.io/python/python_plotly/
- https://www.kaggle.com/gurarako/plotly-radar-chart-pokemon-part-3
- https://bbs.ruliweb.com/family/4749/board/100051/read/9488903
- https://towardsdatascience.com/how-to-create-a-plotly-visualization-and-embed-it-on-websites-517c1a78568b